In [15]:
import numpy as np
import pandas as pd
import scipy.stats
from scipy.stats import rankdata, t, norm
import math

x = np.array([8,6,2,4,7,5,1,3])
y = np.array([7,6,4,2,8,5,3,1])
confidence_level = 0.95


def calculate_p_value_from_z_score(score):
    p_value = scipy.stats.t.sf((abs(score)), 100000) * 2
    return min(float(p_value), 0.99999)


def Spearman_FootRule(x,y, confidence_level):
    Data = pd.DataFrame({'x': x, 'y': y})
    x_ranked, y_ranked = rankdata(x), rankdata(y)

    Sum_of_the_Difference = np.sum(np.abs(x_ranked - y_ranked))
    sample_size = len(x)
    Spearman_Foot_Rule = 1 - 3 * Sum_of_the_Difference / (sample_size**2 - 1)
    Standard_Error = np.sqrt((2 * sample_size**2 + 7) / (5 * (sample_size + 1) * (sample_size - 1)**2))

    # Approximated SIGNIFICANCE AND CONFIDENCE INTERVALS
    Statistic = Spearman_Foot_Rule / Standard_Error
    Approximated_p_Value = calculate_p_value_from_z_score(Statistic)

    zcrit = scipy.stats.t.ppf(1 - (1 - confidence_level) / 2, 100000)
    Lower_ci_Spearman_Foot_Rule = (Spearman_Foot_Rule - zcrit * Standard_Error)
    Upper_ci_Spearman_Foot_Rule = (Spearman_Foot_Rule + zcrit * Standard_Error)

    # Exact Significance - The p_value exact method is based on Berry et al., 2021
    observed_difference = np.sum(np.abs(Data['x'] - Data['y']))
    Spearman_Foot_Statistic = 1 - 3 * observed_difference / (sample_size**2 - 1)
    adjusted_difference = observed_difference + 0.000001

    def permutations_matrix(X):
        return np.array([[1]]) if X == 1 else np.vstack([np.column_stack((np.full((r := permutations_matrix(X - 1)).shape[0], i), r + (r >= i))) for i in range(1, X + 1)])

    permutation_matrix, Counte_Cases_Larger = permutations_matrix(sample_size), 0
    for i in range(math.factorial(sample_size)):
        current_difference = np.sum(np.abs(Data['x'] - permutation_matrix[i, :]))
        Counte_Cases_Larger += 1 if current_difference <= adjusted_difference else 0

    p_value_exact = Counte_Cases_Larger / math.factorial(sample_size)
    
    results= {}

    results["Spearman Footrule"]= Spearman_Foot_Statistic
    results["Exact p_value"] = p_value_exact
    results["Statistic"]= Statistic
    results["Standard_Error_H0"]= Standard_Error
    results["Approximated_p_value"] = Approximated_p_Value
    results["Confidence Intervals Spearman Footrule"] = f"({round(Lower_ci_Spearman_Foot_Rule, 4)}, {round(Upper_ci_Spearman_Foot_Rule, 4)})"

    result_str = "\n".join([f"{key}: {value}" for key, value in results.items()])
    return results

Spearman_FootRule(x,y,0.95)






{'Spearman Footrule': 0.5238095238095238,
 'Exact p_value': 0.030952380952380953,
 'Statistic': 2.116950987028628,
 'Standard_Error_H0': 0.24743582965269675,
 'Approximated_p_value': 0.03426647009130634,
 'Confidence Intervals Spearman Footrule': '(0.0388, 1.0088)'}